## Notebook para hacer el merge de varias imagenes tiff (imagenes pequeñas)

In [100]:
import wx
import cv2
import numpy as np

In [101]:
def get_paths(wildcard):
    app = wx.App(None)
    style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST | wx.MULTIPLE
    dialog = wx.FileDialog(None, 'Open', wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        paths = dialog.GetPaths()
    else:
        paths = None
    dialog.Destroy()
    return paths

In [116]:
filepaths = get_paths('*.tif')

images = []
for file_item in filepaths:
    images.append(cv2.imread(file_item , cv2.IMREAD_UNCHANGED))

In [117]:
# Esta cuenta asume que todas las imagenes cargadas tienen las mismas dimensiones 
# hacer el primer cuadrado mas grande que los demas

N = len(images)

shapes = []
for image in images:
    shapes.append(image.shape)

im_max_y = 0
im_max_x = 0
for im_shape in shapes:
    if im_shape[0] > im_max_y:
        im_max_y = im_shape[0]
    if im_shape[1] > im_max_x:
        im_max_x = im_shape[1]

ly, lx, lz = im_max_y, im_max_x,images[0].shape[2]

image_type = images[0].dtype

A4_ratio = 1.414

nx = np.round(np.sqrt(N*ly/lx*A4_ratio))#.astype(np.int8)

ny = np.ceil(N/nx)

# Crea una imagen en blanco (donde se acomodaran las imagenes cargadas) con una distribución que se asemeja a una hoja A4
canvas = np.zeros([np.int32(ny*ly),np.int32(nx*lx), lz], dtype = image_type)

In [118]:
i = 0
j = 0
n_x = 0
n_y = 0

for image in images:
    
    j = n_x*lx
    i = n_y*ly
    
    lx_image = image.shape[1]
    ly_image = image.shape[0]
    
    canvas[i:i+ly_image, j:j+lx_image, :] = image
       
    n_x += 1
       
    if not n_x%nx:
        n_y +=1
        n_x = 0

In [119]:
cv2.imshow('Merged Images', canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [120]:
path_chunks = filepaths[0].split('\\')

path_only = path_chunks[0]
for chunk in range(1,len(path_chunks)-1):
    path_only = path_only +'/'+ path_chunks[chunk]

print path_only

C:/Users/Roy/Desktop/Exp4/Datos/Tailandia/Ramathibodi/film calibration


In [121]:
cv2.imwrite(path_only+'/'+'merged_calibration.tif', canvas)

True